In [1]:
import pandas as pd
import numpy as np
import re
import os
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('Resume.csv')

In [3]:
df.head()

,field,Resume
0,Peoplesoft Resume,Anubhav Kumar Singh\t\t To work in a global...
1,Peoplesoft Resume,\tProfile Summary:\t\t\t\t\t\t\t\t\t\t7+ years...
2,Peoplesoft Resume,PeopleSoft Database Administrator ...
3,Peoplesoft Resume,MuraliExperience Summary I have 6 years of exp...
4,Peoplesoft Resume,"Priyanka Ramadoss61/46, MountPleasant, Coonoor..."


In [4]:
df.rename(columns = {'field' : 'Category'},inplace = True)

In [5]:
df['Category'][0]

'Peoplesoft Resume'

In [6]:

def cleanresume(text):
    text = re.sub('http\S+\s',' ',text) #removing urls
    text = re.sub('#\S+','',text)    #removing hasgtags
    text = re.sub('@\S+','  ',text)   #Removing '@' from text
    text = re.sub("[0-9" "]+"," ",text) #removing numbers
    text = re.sub('[%s]' % re.escape("""!"#S%&'()*+,-./:
    ;<=>?@[\]^_`{|}~"""), ' ',text)
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    text = re.sub('\s+',' ',text)
    text = re.sub('[^A-Za-z0-9 ]|_|-',' ',text)
    
    return text

In [7]:
df['cleaned_resume'] = df.Resume.apply(lambda x : cleanresume(x))
print(df['cleaned_resume'])

0      Anubhav Kumar ingh To work in a globally comp...
1      Profile ummary years of experience in impleme...
2     People oft Database Administrator Gangareddy P...
3     MuraliExperience ummary I have years of experi...
4     Priyanka Ramadoss MountPleasant CoonoorThe Nil...
                            ...                        
74     Workday Integration ConsultantName ri Krishna...
75     eeking suitable positions in Workday HCM as T...
76    WORKDAY HCM FCMName Kumar Role Workday Consult...
77    Venkateswarlu B Workday Consultant Having year...
78     Vinay kumar v Workday Functional ConsultantEX...
Name: cleaned_resume, Length: 79, dtype: object


In [8]:
df['cleaned_resume'][1]

' Profile ummary years of experience in implementing upgrading and supporting People oft database administration including Human Capital Management HCM Financials Campus olutions and Portal IHUBExpertise in installation configuration setup security and management of the People oft Internet Architecture PIA environment In depth experience in analysis planning development and implementation stages including load testing quality assurance and tuning Gained extensive exposure in deploying People oft Environments Experienced in Troubleshooting People oft Components killed with the capability to analyse interpret unique problems with a combination of training experience logical thinking to find the right solutionsCore Competencies People oft Implementation Troubleshooting People oft performance issues Project Data Migration Installation Configuration of People oft components PUM DPK Install and configure Upgrades People tools Expertise in applying Patches and Updates via Change Assistant too

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


In [10]:
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

In [11]:
df.Category.unique()

array([0, 1, 2, 3])

In [12]:
df.drop('Resume',axis = 1,inplace = True)

In [13]:
df

,Category,cleaned_resume
0,0,Anubhav Kumar ingh To work in a globally comp...
1,0,Profile ummary years of experience in impleme...
2,0,People oft Database Administrator Gangareddy P...
3,0,MuraliExperience ummary I have years of experi...
4,0,Priyanka Ramadoss MountPleasant CoonoorThe Nil...
...,...,...
74,3,Workday Integration ConsultantName ri Krishna...
75,3,eeking suitable positions in Workday HCM as T...
76,3,WORKDAY HCM FCMName Kumar Role Workday Consult...
77,3,Venkateswarlu B Workday Consultant Having year...


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer(stop_words='english')

vector.fit(df['cleaned_resume'])
required_text = vector.transform(df['cleaned_resume'])

In [15]:
df

,Category,cleaned_resume
0,0,Anubhav Kumar ingh To work in a globally comp...
1,0,Profile ummary years of experience in impleme...
2,0,People oft Database Administrator Gangareddy P...
3,0,MuraliExperience ummary I have years of experi...
4,0,Priyanka Ramadoss MountPleasant CoonoorThe Nil...
...,...,...
74,3,Workday Integration ConsultantName ri Krishna...
75,3,eeking suitable positions in Workday HCM as T...
76,3,WORKDAY HCM FCMName Kumar Role Workday Consult...
77,3,Venkateswarlu B Workday Consultant Having year...


In [16]:
required_text

<79x4550 sparse matrix of type '<class 'numpy.float64'>'
	with 18050 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(required_text, df['Category'], test_size=0.2, random_state=42)

In [18]:
X_train.shape

(63, 4550)

In [19]:
y_train.shape

(63,)

In [20]:
y_test.shape

(16,)

In [21]:
X_test.shape

(16, 4550)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train,y_train)
ypred = clf.predict(X_test)
print(accuracy_score(y_test,ypred))

1.0


In [23]:
ypred

array([1, 0, 1, 1, 0, 1, 0, 3, 0, 0, 2, 1, 3, 1, 3, 2])

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Dictionary containing classifiers
classifiers = {
    "Multinomial Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC()
}

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(required_text, df['Category'], test_size=0.3, random_state=42)

# Iterate over classifiers
for name, classifier in classifiers.items():
    print(f"Evaluating {name}...")
    
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions on the test set
    predictions = classifier.predict(X_test)
    
    # Evaluate the classifier
    report = classification_report(y_test, predictions)
    print(report)
    print("="*60)


Evaluating Multinomial Naive Bayes...
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         6

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24

Evaluating Logistic Regression...
              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       0.88      1.00      0.93         7
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         6

    accuracy                           0.96        24
   macro avg       0.97      0.96      0.96        24
weighted avg       0.96      0.96      0.96        24

Evaluating Random Forest...
              precision    rec

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(required_text, df['Category'], test_size=0.2, random_state=42)

# Initialize the Logistic Regression classifier
logistic_regression = LogisticRegression()

# Train the classifier
logistic_regression.fit(X_train, y_train)

# Make predictions on the test set
predictions = logistic_regression.predict(X_test)

# Print predicted labels alongside actual labels
print("Predicted labels:", predictions)
print("Actual labels:", y_test.values)

# Evaluate the classifier
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)


Predicted labels: [1 0 1 1 1 1 0 3 0 0 2 1 3 1 3 2]
Actual labels: [1 0 1 1 0 1 0 3 0 0 2 1 3 1 3 2]
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.86      1.00      0.92         6
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         3

    accuracy                           0.94        16
   macro avg       0.96      0.95      0.95        16
weighted avg       0.95      0.94      0.94        16



In [28]:
import pickle

In [29]:
with open("Model.pkl","wb") as file:
    pickle.dump(logistic_regression,file)


In [30]:
pickle.dump(vector,open('similarity.pkl','wb'))

In [31]:
pickle.dump(logistic_regression,open('logistic.pkl','wb'))